# Chains

In [1]:
import os
import openai

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
openai.api_key = os.environ["OPENAI_API_KEY"]

In [2]:
import pandas as pd

df = pd.read_csv("Data.csv")
df

,Product,Review
0,Queen Size Sheet Set,I ordered a king size set. My only criticism w...
1,Waterproof Phone Pouch,"I loved the waterproof sac, although the openi..."
2,Luxury Air Mattress,This mattress had a small hole in the top of i...
3,Pillows Insert,This is the best throw pillow fillers on Amazo...
4,Milk Frother Handheld\n,I loved this product. But they only seem to l...
5,L'Or Espresso Café \n,Je trouve le goût médiocre. La mousse ne tient...
6,Hervidor de Agua Eléctrico,"Está lu bonita calienta muy rápido, es muy fun..."


## LLMChain

In [3]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

llm = ChatOpenAI(temperature=0.9)
prompt = ChatPromptTemplate.from_template("What is the best name to describe a company that makes {product}?")
chain = LLMChain(llm=llm, prompt=prompt)
chain.invoke({"product": "Queen Size Sheet Set"})

C:\Users\tchak\AppData\Local\Temp\ipykernel_2300\1831501332.py:7: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(llm=llm, prompt=prompt)


{'product': 'Queen Size Sheet Set', 'text': '"Royal Comforts"'}

## SimpleSequentialChain (single input, single output)

In [4]:
from langchain.chains import SimpleSequentialChain

prompt_1 = ChatPromptTemplate.from_template("What is the best name to describe a company that makes {product}?")
chain_1 = LLMChain(llm=llm, prompt=prompt_1)

prompt_2 = ChatPromptTemplate.from_template("Write a 20 words description for the following company:{company_name}")
chain_2 = LLMChain(llm=llm, prompt=prompt_2)

chain = SimpleSequentialChain(chains=[chain_1, chain_2], verbose=True)
chain.invoke({"input": "Queen Size Sheet Set"})



> Entering new SimpleSequentialChain chain...
"Regal Linens"
Regal Linens offers luxurious bed and bath linens crafted from high-quality materials to elevate your home decor and comfort.

> Finished chain.


{'input': 'Queen Size Sheet Set',
 'output': 'Regal Linens offers luxurious bed and bath linens crafted from high-quality materials to elevate your home decor and comfort.'}

## SequentialChain (multiple inputs, multiple outputs)

In [5]:
from langchain.chains import SequentialChain

# 1. input: review, output: english_review
prompt_1 = ChatPromptTemplate.from_template(
    "Translate the following review to english:"
    "\n\n{review}"
)
chain_1 = LLMChain(llm=llm, prompt=prompt_1, output_key="english_Review")

# 2. input: english_review, output: summary
prompt_2 = ChatPromptTemplate.from_template(
    "Can you summarize the following review in 1 sentence:"
    "\n\n{english_Review}"
)
chain_2 = LLMChain(llm=llm, prompt=prompt_2, output_key="summary")

# 3. input: review, output: language
prompt_3 = ChatPromptTemplate.from_template(
    "What language is the following review:"
    "\n\n{review}"
)
chain_3 = LLMChain(llm=llm, prompt=prompt_3, output_key="language")

# 4. input: summary + language, output: followup_message 
prompt_4 = ChatPromptTemplate.from_template(
    "Write a follow up response to the following summary in the specified language:"
    "\n\nSummary: {summary}\n\nLanguage: {language}"
)
chain_4 = LLMChain(llm=llm, prompt=prompt_4, output_key="followup_message")

# Overall
chain = SequentialChain(
    chains=[chain_1, chain_2, chain_3, chain_4],
    input_variables=["review"],
    output_variables=["english_Review", "summary", "language", "followup_message"],
    verbose=True,
)
chain.invoke({"review": df["Review"][5]})



> Entering new SequentialChain chain...

> Finished chain.


{'review': "Je trouve le goût médiocre. La mousse ne tient pas, c'est bizarre. J'achète les mêmes dans le commerce et le goût est bien meilleur...\nVieux lot ou contrefaçon !?",
 'english_Review': "I find the taste mediocre. The foam doesn't hold, it's weird. I buy the same ones in stores and the taste is much better... Old batch or counterfeit!?",
 'summary': "The reviewer found the taste of the product to be mediocre and noticed that the foam didn't hold, leading them to question if it was an old batch or counterfeit.",
 'language': 'French',
 'followup_message': "Je suis désolé d'entendre que vous avez trouvé le goût du produit moyen et que la mousse ne tenait pas. Il est possible que ce soit un vieux lot ou une contrefaçon. Je vous recommande de contacter le fabricant pour clarifier la situation. J'espère que vous pourrez résoudre ce problème et trouver un produit de meilleure qualité à l'avenir."}